In [4]:
from mnist_loader import load_data_wrapper
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import time

In [5]:
dane_treningowe, dane_walidacyjne, dane_testowe = load_data_wrapper()

In [6]:
# Musimy unormować dane
def normuj_dane(dane):
    unormowane = []
    for  x, y in dane:
        x_norm = x/255
        unormowane.append((x_norm,y))
    return unormowane

In [9]:
dane_treningowe = normuj_dane(dane_treningowe) # normlaizujemy dane
dane_walidacyjne = normuj_dane(dane_walidacyjne)
dane_testowe = normuj_dane(dane_testowe) # normalizujemy dane

print(len(dane_treningowe)) 
print(len(dane_walidacyjne))
print(len(dane_testowe)) 

50000
10000
10000


#### Definiujemy byty

* Warstwa, metody:
1. forward propagation
2. back propagation

 Warstwa powinna mieć atrybuty:
- wagi
- dane_na_wejsciu (zapamietać)
- dane_na_wyjsciu (przekazujemy)
- liczba neuronow
- funkcja aktywacji
- net (zapamiętać)

Zapamiętać np. w liście

* Sieć (składa sie z wielu Warstw), metody
1. forward prop - wykonujemy na każdej z warstw
2. backward prop - wykonujemy na każdej z warstw
3. fit
4. predict
5. dokladnosc

* Czy dobre byłoby także znormalizować dane, aby przyjmowały wartości z przedziału [0,1]?
 W ten sposób zminimalizujemy ryzyko liczenia gradientów o bardzo dużych wartościach. Pytanie czy wtedy trzeba generować dane z rozkladu $\mathcal{N}(0,1/\sqrt{n})$, n - liczba danych na wejściu
 Czy wystarczy $\mathcal{N}(0,1)$

 * Czy aktualizację wag lepiej jest robić w osobnej funkcji w klasie Warstwa, czy podczas back_prop?

In [17]:
def wygeneruj_wagi(wymiar_wejscie, wymiar_wyjscie):
    # wyjściem jest liczba neuronów w następnej warstwie
    wektor_wag = np.random.normal(0,1/math.sqrt(wymiar_wejscie),(1+wymiar_wejscie)*wymiar_wyjscie)
    return np.reshape(wektor_wag, (wymiar_wyjscie, 1+wymiar_wejscie )  )

# Definiujemy funkcje aktywacji fi (sigmoid)
# fi' = fi*(1-fi) 
def sigmoid(x): return (1+np.exp(-x))**(-1) 

def deriv_sigmoid(x): return sigmoid(x)*(1-sigmoid(x))

In [ ]:
class SiecNeuronowa:
    def __init__(self, wymiary = [784, 128, 64, 10]):
        # definiujemy listę obiektów warstwy
        self.warstwy = list()
        # Może każdej warstwy definiujemy funkcję aktywacji? (W ten sposób można wykorzystać różne funkcje)

        for i in range(len(wymiary)-1):
            warstwa = Warstwa(wymiary[i+1], wymiary[i])
            self.warstwy.append(warstwa)

    def forward_propagation(self, X):
        wyjscie = X
        
        for warstwa in self.warstwy:
            wyjscie = warstwa.forward_prop(wyjscie)
        return wyjscie
            

    def backward_propagation(self):
        pass
        
    def fit(self):
        pass

    def predict(self):
        pass

    def dokladnosc(self):
        pass

class Warstwa:
    def __init__(self, liczba_neuronow, dane_wejscie):
        """liczba neuronów - w warstwie (czyli liczba danych na wyjściu)"""
        self.liczba_neuronow = liczba_neuronow

        self.dane_wejscie = dane_wejscie
        self.liczba_wejscie = len(self.dane_wejscie)
        # Ustalamy stałą uczenia dla całej warstwy
        self.stala_uczenia = 0.1

        # Inicjalizacja wag
        self.wagi = wygeneruj_wagi(self.dane_wejscie, self.liczba_neuronow)

        # Rzeczy do zapamiętania
        self.lista_dane_wejscie = list()
        self.lista_net = list()
    

    def fun_aktywacji(self, x): return sigmoid(x)

    def pochodna_fun_aktywacji(self, x): return deriv_sigmoid(x)

    def forward_prop(self, dane_wejscie ):
        """Definiujemy propagację w przód.
        Podajemy dane na wejście, dodajemy na bias = 1 na początek wektora (otrzymujemy w ten sposób X).
        Zadajemy najpierw ile chcemy mieć neuronów na wyjściu
        Nastepnie liczymy net = W*X <- wyjście netto.
        Póżniej nakładamy funkcję aktywacji fi na net, otrzymując fi(net) = a <- wyjście z warstwy.
        Musimy też zapisywać stany x, net, a dla każdej warstwy (np. listy)"""

        # Musimy przerobić X na wektor kolumnowy
        self.X = np.vstack([1, dane_wejscie.reshape(-1,1)])
        self.net = self.wagi @ self.X
        dane_wyjscie = self.fun_aktywacji(self.net)

        self.lista_net.append(self.net)
        self.lista_dane_wejscie.append(self.X)

        self.wyjscia_forward_prop = dane_wyjscie

        return dane_wyjscie
    
    def back_prop(self, wyjscie_oczekiwane):
        """Definiujemy propagację wstecz.
        W ostatniej warstwie liczymy funkcję straty, następnie pochodną dL/da."""
        
        # Funkcja straty to L = 1/2(a[L]-y)^2, czyli pochodna z L to a[L]-y
        dL_a = self.wyjscia_forward_prop - wyjscie_oczekiwane

        # mamy pochodną dL/da * fi(net)
        delta = dL_a * self.pochodna_fun_aktywacji( self.net )
        dL_dW = delta @ self.X.T

        # Liczymy dL/dX i usuwamy pierwszy element, otrzymując dL/da niższej warstwy
        dL_dX = self.wagi.T @ delta
        
        # Dane do przekazania warstwę niżej
        dL_a = dL_dX[1:]

        # Aktualizujemy wagi
        self.wagi = self.wagi - self.stala_uczenia * dL_dW

        return dL_a



In [21]:
x = np.array([1,0]).reshape(2,1)
y = np.array([0,1]).reshape(2,1)
W1 = np.array([[-0.1, -0.2, 0.1],
               [-0.4, 0.7, -0.6]])
W2 = np.array([[-0.15, -0.25, 0.15],
               [-0.45, 0.75, -0.65]])
X1 = np.vstack((1,x))


#### Lepiej będzie chyba podawać w `__init__` Warstwa liczbę danych na wejściu i liczbę neuronów

In [68]:
class SiecNeuronowa:
    def __init__(self, wymiary = [784, 128, 64, 10]):
        # definiujemy listę obiektów warstwy
        self.warstwy = list()
        # Może każdej warstwy definiujemy funkcję aktywacji? (W ten sposób można wykorzystać różne funkcje)

        for i in range(len(wymiary)-1):
            warstwa = Warstwa(wymiary[i+1], wymiary[i])
            self.warstwy.append(warstwa)

    def forward_propagation(self, X):
        wyjscie = X
        
        for warstwa in self.warstwy:
            wyjscie = warstwa.forward_prop(wyjscie)
            print("wyjscie z warstwy")
            print(wyjscie)
        return wyjscie
            

    def backward_propagation(self):
        pass
        
    def fit(self):
        pass

    def predict(self):
        pass

    def dokladnosc(self):
        pass

class Warstwa:
    def __init__(self, liczba_wejscie, liczba_neuronow):
        """liczba neuronów - w warstwie (czyli liczba danych na wyjściu)"""
        self.liczba_neuronow = liczba_neuronow

        self.liczba_wejscie = liczba_wejscie
        
        # Ustalamy stałą uczenia dla całej warstwy
        self.stala_uczenia = 0.1

        # Inicjalizacja wag
        self.wagi = wygeneruj_wagi(self.liczba_wejscie, self.liczba_neuronow)

        # Rzeczy do zapamiętania
        self.lista_dane_wejscie = list()
        self.lista_net = list()
    

    def fun_aktywacji(self, x): return sigmoid(x)

    def pochodna_fun_aktywacji(self, x): return deriv_sigmoid(x)

    def forward_prop(self, dane_wejscie ):
        """Definiujemy propagację w przód.
        Podajemy dane na wejście, dodajemy na bias = 1 na początek wektora (otrzymujemy w ten sposób X).
        Zadajemy najpierw ile chcemy mieć neuronów na wyjściu
        Nastepnie liczymy net = W*X <- wyjście netto.
        Póżniej nakładamy funkcję aktywacji fi na net, otrzymując fi(net) = a <- wyjście z warstwy.
        Musimy też zapisywać stany x, net, a dla każdej warstwy (np. listy)"""

        # Musimy przerobić X na wektor kolumnowy
        self.X = np.vstack([1, dane_wejscie.reshape(-1,1)])
        self.net = self.wagi @ self.X
        dane_wyjscie = self.fun_aktywacji(self.net)

        self.lista_net.append(self.net)
        self.lista_dane_wejscie.append(self.X)

        self.wyjscia_forward_prop = dane_wyjscie

        return dane_wyjscie
    
    def back_prop(self, wyjscie_oczekiwane):
        """Definiujemy propagację wstecz.
        W ostatniej warstwie liczymy funkcję straty, następnie pochodną dL/da."""
        
        # Funkcja straty to L = 1/2(a[L]-y)^2, czyli pochodna z L to a[L]-y

        # Ten fragment chyba nie jest potrzeby
        dL_a = wyjscie_oczekiwane
        #dL_a = self.wyjscia_forward_prop - wyjscie_oczekiwane
        #print("dL_a")
        #print(dL_a)

        # mamy pochodną dL/da * fi(net)
        delta = dL_a * self.pochodna_fun_aktywacji( self.net )
        print("delta")
        print(delta)

        dL_dW = delta @ self.X.T
        print("dL_dW")
        print(dL_dW)

        # Liczymy dL/dX i usuwamy pierwszy element, otrzymując dL/da niższej warstwy
        dL_dX = self.wagi.T @ delta
        
        # Dane do przekazania warstwę niżej
        dL_a = dL_dX[1:]

        # Aktualizujemy wagi
        self.wagi = self.wagi - self.stala_uczenia * dL_dW

        return dL_a



In [ ]:
x = np.array([1,0]).reshape(2,1)
y = np.array([0,1]).reshape(2,1)
W1 = np.array([[-0.1, -0.2, 0.1],
               [-0.4, 0.7, -0.6]])
W2 = np.array([[-0.15, -0.25, 0.15],
               [-0.45, 0.75, -0.65]])
X1 = np.vstack((1,x))

warstwa1 = Warstwa(2,2)
warstwa1.wagi = W1
print("a[1]")
print(warstwa1.forward_prop(x)) # dostajemy a[1] z przykładu ML_06
a1 = warstwa1.forward_prop(x)


warstwa2 = Warstwa(2,2)
warstwa2.wagi = W2
#print(warstwa2.wagi)
a2 = warstwa2.forward_prop(a1)
print("a[2]")
print(a2)

# Czyli forward_prop działa
# Sprawdźmy back_prop

#w back_prop podajemy dL_da[k], otrzymując w ten sposób dL_da[k-1] z niższej warstwy (w międzyczasie aktualizujemy wagi)
dL_da2 = a2 - y
print("dL_da2")
print(dL_da2)
print()

dL_da1 = warstwa2.back_prop(dL_da2)
print("dL_da1")
print(dL_da1)
warstwa2.lista_net #mamy net1

print("nowe_wagi dla warstwy 2")
print(warstwa2.wagi)
############################## Kolejna warstwa ##########################

print("Obliczenia dla warstwy niżej")
print()

dL_da0 = warstwa1.back_prop(dL_da1)
print("dL_da0")
print(dL_da0)

print("nowe wagi dla warstwy 1")
print(warstwa1.wagi)
# Zwraca poprawne wyniki
# Czyli forward i back prop są dobrze zdefiniowane


a[1]
[[0.42555748]
 [0.57444252]]
a[2]
[[0.45754671]
 [0.37654958]]
dL_da2
[[ 0.45754671]
 [-0.62345042]]

delta
[[ 0.11356205]
 [-0.14636122]]
dL_dW
[[ 0.11356205  0.04832718  0.06523487]
 [-0.14636122 -0.06228511 -0.08407611]]
dL_da1
[[-0.13816143]
 [ 0.1121691 ]]
nowe_wagi dla warstwy 2
[[-0.16135621 -0.25483272  0.14347651]
 [-0.43536388  0.75622851 -0.64159239]]
Obliczenia dla warstwy niżej

delta
[[-0.03377471]
 [ 0.02742067]]
dL_dW
[[-0.03377471 -0.03377471  0.        ]
 [ 0.02742067  0.02742067  0.        ]]
dL_da0
[[ 0.02594941]
 [-0.01982987]]
nowe wagi dla warstwy 1
[[-0.09662253 -0.19662253  0.1       ]
 [-0.40274207  0.69725793 -0.6       ]]
